<a href="https://colab.research.google.com/github/SaideLeon/ClonarVOZ/blob/main/Qwen3_TTS_Clone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# Qwen3-TTS — Clonagem de Voz com Interface em Português
#
# Modelo: Qwen3-TTS-12Hz-1.7B-Base
# Função: Voice Clone (Clonagem de Voz)
# ============================================

# ============================================
# CÉLULA 0: Configuração do ambiente
# ============================================
# Este notebook salva perfis localmente (sessão do Colab) em /content.

# ============================================
# CÉLULA 1: Instalação das dependências
# ============================================
!pip install -q qwen-tts flash-attn --no-build-isolation gradio soundfile

# ============================================
# CÉLULA 2: Importação das bibliotecas
# ============================================
import torch
import soundfile as sf
import gradio as gr
from qwen_tts import Qwen3TTSModel
import os # Added for persistence
import json # Added for persistence
import shutil # Added for persistence

# ============================================
# CÉLULA 3: Configuração da persistência (local)
# ============================================
BASE_DIR = "/content/qwen_tts_perfis"
os.makedirs(BASE_DIR, exist_ok=True)

# ============================================
# CÉLULA 4: Carregamento do modelo de clonagem
# ============================================
print("Carregando modelo de clonagem de voz...")
model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-1.7B-Base",
    device_map="cuda:0",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)
print("Modelo carregado com sucesso!")

# ============================================
# CÉLULA 5: Funções de persistência de perfis
# ============================================

def salvar_perfil_local(nome, audio_path, texto_ref, idioma):
    perfil_dir = os.path.join(BASE_DIR, nome)
    os.makedirs(perfil_dir, exist_ok=True)

    # áudio
    audio_dest = os.path.join(perfil_dir, "referencia.wav")
    shutil.copy(audio_path, audio_dest)

    # texto
    with open(os.path.join(perfil_dir, "texto.txt"), "w", encoding="utf-8") as f:
        f.write(texto_ref)

    # metadados
    meta = {"idioma": idioma}
    with open(os.path.join(perfil_dir, "meta.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    return f"Perfil '{nome}' salvo com sucesso!"

def carregar_perfis_local():
    perfis = {} # {nome: {"audio_path": ..., "texto_ref": ..., "idioma": ...}}

    if not os.path.exists(BASE_DIR):
        return perfis

    for nome in os.listdir(BASE_DIR):
        perfil_dir = os.path.join(BASE_DIR, nome)
        if not os.path.isdir(perfil_dir):
            continue

        try:
            audio_path = os.path.join(perfil_dir, "referencia.wav")
            with open(os.path.join(perfil_dir, "texto.txt"), "r", encoding="utf-8") as f:
                texto_ref = f.read()
            with open(os.path.join(perfil_dir, "meta.json"), "r", encoding="utf-8") as f:
                meta = json.load(f)
            idioma = meta.get("idioma", "Auto")

            if os.path.exists(audio_path) and texto_ref.strip(): # Ensure text is not empty
                perfis[nome] = {
                    "audio_path": audio_path,
                    "texto_ref": texto_ref,
                    "idioma": idioma
                }
            else:
                print(f"Aviso: Perfil '{nome}' incompleto (falta áudio ou texto de referência). Ignorando.")

        except Exception as e:
            print(f"Erro ao carregar perfil '{nome}': {e}")
    return perfis

# Load profiles initially when the notebook runs
perfis_carregados = carregar_perfis_local()


# ============================================
# CÉLULA 6: Função de clonagem de voz
# ============================================
def clonar_voz(
    novo_texto,
    idioma,
    audio_referencia_path, # This is a path now
    texto_referencia
):
    """
    Clona a voz a partir de um áudio de referência
    """

    # Validações
    if not novo_texto.strip():
        return None, "❌ Insira o texto que será falado"

    if audio_referencia_path is None or not os.path.exists(audio_referencia_path):
        return None, "❌ Envie um áudio de referência ou selecione um perfil válido."

    if not texto_referencia.strip():
        return None, "❌ Informe o texto falado no áudio de referência"

    try:
        # Geração da voz clonada
        wavs, sr = model.generate_voice_clone(
            text=novo_texto,
            language=idioma if idioma != "Auto" else "Auto",
            ref_audio=audio_referencia_path,
            ref_text=texto_referencia,
        )

        # Retorna tupla (array_audio, sample_rate)
        return (sr, wavs[0]), "✅ Voz clonada com sucesso!"

    except Exception as e:
        return None, f"❌ Erro: {str(e)}"

# ============================================
# CÉLULA 7: Interface Gradio (100% em português)
# ============================================
with gr.Blocks(title="Clonagem de Voz — Qwen3-TTS") as demo:
    gr.Markdown("# 🎙️ Clonagem de Voz com IA (Qwen3-TTS)")
    gr.Markdown(
        "Clone uma voz real a partir de um áudio curto e gere novas falas com alta fidelidade."
    )

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 🗂️ Gerenciamento de Perfis de Voz")
            perfil_selecionado = gr.Dropdown(
                choices=list(perfis_carregados.keys()),
                label="Selecionar Perfil de Voz Salvo",
                info="Selecione um perfil salvo para carregar suas informações."
            )
            nome_novo_perfil = gr.Textbox(
                label="Nome para Salvar Novo Perfil",
                placeholder="Ex: Voz do João, Voz da Maria...",
                info="Este nome será usado para salvar o novo perfil localmente."
            )
            btn_salvar_perfil = gr.Button(
                "💾 Salvar Perfil Atual",
                variant="secondary"
            )
            status_perfil = gr.Textbox(label="Status do Perfil", interactive=False)


            gr.Markdown("---") # Separator

            # Original inputs for cloning
            novo_texto = gr.Textbox(
                label="📝 Texto a ser falado",
                placeholder="Digite aqui o novo texto que a voz clonada irá falar...",
                lines=4
            )

            idioma = gr.Dropdown(
                choices=[
                    "Auto", "Chinese", "English", "Japanese", "Korean",
                    "German", "French", "Russian", "Portuguese", "Spanish", "Italian"
                ],
                value="Auto",
                label="🌍 Idioma"
            )

            gr.Markdown("### 🎧 Áudio de referência (voz original)")

            audio_referencia = gr.Audio(
                label="Áudio de referência",
                type="filepath",
                # Pre-fill if profiles exist
                value=perfis_carregados[list(perfis_carregados.keys())[0]]["audio_path"] if perfis_carregados else None
            )

            texto_referencia = gr.Textbox(
                label="📄 Texto falado no áudio",
                placeholder="Digite exatamente o que é falado no áudio enviado",
                lines=3,
                # Pre-fill if profiles exist
                value=perfis_carregados[list(perfis_carregados.keys())[0]]["texto_ref"] if perfis_carregados else ""
            )

            gerar_btn = gr.Button(
                "🎤 Clonar voz e gerar áudio",
                variant="primary",
                size="lg"
            )

        with gr.Column():
            audio_saida = gr.Audio(
                label="🔊 Áudio gerado",
                type="numpy"
            )
            status = gr.Textbox(
                label="Status da Geração",
                lines=2,
                interactive=False
            )

            # Gradio event handlers
            gerar_btn.click(
                fn=clonar_voz, # Direct call to clonar_voz
                inputs=[novo_texto, idioma, audio_referencia, texto_referencia],
                outputs=[audio_saida, status]
            )

            def _carregar_perfil_ui(selected_profile_name):
                if selected_profile_name and selected_profile_name in perfis_carregados:
                    profile_data = perfis_carregados[selected_profile_name]
                    return profile_data["audio_path"], profile_data["texto_ref"], profile_data["idioma"], f"Perfil '{selected_profile_name}' carregado."
                return None, "", "Auto", "Selecione um perfil ou envie novos dados."

            perfil_selecionado.change(
                fn=_carregar_perfil_ui,
                inputs=[perfil_selecionado],
                outputs=[audio_referencia, texto_referencia, idioma, status_perfil]
            )

            # Combined function to save profile and update UI
            def _salvar_perfil_e_atualizar_ui(nome_do_perfil, audio_ref_path, texto_ref_input, idioma_input):
                global perfis_carregados # Important to update global state
                if not nome_do_perfil.strip():
                    return "❌ Forneça um nome para o perfil.", gr.Dropdown.update(choices=list(perfis_carregados.keys()), value=None)
                if audio_ref_path is None or not os.path.exists(audio_ref_path): # Check for actual file path
                    return "❌ Envie um áudio de referência para salvar.", gr.Dropdown.update(choices=list(perfis_carregados.keys()), value=None)
                if not texto_ref_input.strip():
                    return "❌ Forneça o texto falado no áudio de referência para salvar.", gr.Dropdown.update(choices=list(perfis_carregados.keys()), value=None)

                current_status_message = ""
                updated_dropdown_choices = list(perfis_carregados.keys())
                selected_dropdown_value = None

                try:
                    # Attempt to save the profile
                    current_status_message = salvar_perfil_local(nome_do_perfil, audio_ref_path, texto_ref_input, idioma_input)
                    selected_dropdown_value = nome_do_perfil # Assume success for dropdown value
                except Exception as e:
                    return f"❌ Erro ao salvar o perfil: {str(e)}", gr.Dropdown.update(choices=updated_dropdown_choices, value=selected_dropdown_value)

                # If saving was successful, try to reload profiles
                try:
                    perfis_carregados = carregar_perfis_local()
                    updated_dropdown_choices = list(perfis_carregados.keys())
                except Exception as e:
                    # If reloading fails, append this error to the success message
                    current_status_message += f"\n⚠️ Atenção: Perfil salvo, mas houve um erro ao recarregar a lista de perfis: {str(e)}"
                    # Ensure the newly saved profile is in the list of choices, even if carregar_perfis_local failed
                    if nome_do_perfil not in updated_dropdown_choices:
                        updated_dropdown_choices.append(nome_do_perfil)
                updated_dropdown_choices.sort() # Keep it sorted

                return current_status_message, gr.Dropdown.update(choices=updated_dropdown_choices, value=selected_dropdown_value)

            btn_salvar_perfil.click(
                fn=_salvar_perfil_e_atualizar_ui,
                inputs=[nome_novo_perfil, audio_referencia, texto_referencia, idioma],
                outputs=[status_perfil, perfil_selecionado]
            )

    # ============================================
    # Inicialização da aplicação
    # ============================================
    demo.launch(share=True, debug=False)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 99.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 106.8 MB/s eta 0:00:00
